In [1]:
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

from collections import defaultdict

import os
import os.path as osp

In [2]:
import re
stream_re = re.compile(r"(\w+)_\w+.npz")

index_file = "./data/Cert_136033-149442_7TeV_Apr21ReReco_Collisions10_JSON_v2.txt"

columns = [
    'run', 'lumiBlock', 'timeHigh', 'timeLow',
    'jet_pt', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_fX', 'yet_fY', 'yet_fZ',
    'pho_pt', 'pho_eta', 'pho_phi',             'pho_fX', 'pho_fY', 'pho_fZ',
    'muo_pt', 'muo_eta', 'muo_phi', 'muo_mass', 'muo_fX', 'muo_fY', 'muo_fZ',
    'instantLumi'
]

features = [
    'jet_pt', 'jet_eta', 'jet_phi', 'jet_mass', 'jet_fX', 'yet_fY', 'yet_fZ',
    'pho_pt', 'pho_eta', 'pho_phi',             'pho_fX', 'pho_fY', 'pho_fZ',
    'muo_pt', 'muo_eta', 'muo_phi', 'muo_mass', 'muo_fX', 'muo_fY', 'muo_fZ',
    'instantLumi'
]

In [3]:
def read(path='./data/', target_stream = 'minibias', columns = None):
    df = pd.DataFrame()

    get_stream = lambda item: stream_re.findall(item)[0]
    
    streams_data = list()
    
    for item in [ x for x in os.listdir(path) if x.endswith('.npz') ]:
        stream = get_stream(item)
        if stream != target_stream:
            continue

        print 'Reading', item
        
        npz = np.load(osp.join(path, item))

        assert npz.keys() == ['array']
        
        array = npz['array']
        df = pd.DataFrame(array)
        
        streams_data.append(df)
        npz.close()
    
    df = pd.concat(streams_data)

    if columns is not None:
        df.columns = columns

    df['run'] = df['run'].astype('int64')
    df['lumiBlock'] = df['lumiBlock'].astype('int64')
    df['time'] = df['timeHigh'] * 1.0e+6 + df['timeLow']

    df[features] = df[features].astype('float32')

    del df['timeHigh']
    del df['timeLow']

    return df

In [4]:
data = read(columns=columns, target_stream='photon')

Reading photon_ad.npz
Reading photon_ab.npz
Reading photon_ac.npz
Reading photon_aa.npz
Reading photon_ae.npz


In [5]:
data.head()

,run,lumiBlock,jet_pt,jet_eta,jet_phi,jet_mass,jet_fX,yet_fY,yet_fZ,pho_pt,...,pho_fZ,muo_pt,muo_eta,muo_phi,muo_mass,muo_fX,muo_fY,muo_fZ,instantLumi,time
0,149181,1524,48.301361,-1.048085,-2.222497,4.606378,0.088779,0.026352,-2.764762,31.912352,...,-2.764853,0,0,0,0,0,0,0,109.288368,1.288250e+15
1,149181,1524,37.692146,-0.035126,1.910886,5.673314,0.092301,0.016964,1.154524,24.919827,...,1.154382,0,0,0,0,0,0,0,109.288368,1.288250e+15
2,149181,1524,35.128605,-0.063224,-1.417862,3.207675,0.093280,0.013377,2.239151,32.027729,...,2.239970,0,0,0,0,0,0,0,109.288368,1.288250e+15
3,149181,1524,86.766937,0.359164,-3.047801,10.302987,0.097872,0.025091,-0.605730,21.194616,...,-0.605812,0,0,0,0,0,0,0,109.288368,1.288250e+15
4,149181,1524,31.478086,-0.770622,0.833060,4.695371,0.094430,0.017354,-5.551055,25.433561,...,-5.551365,0,0,0,0,0,0,0,109.288368,1.288250e+15


In [6]:
percentiles = [1, 5, 10, 25, 50, 75, 90, 95, 99]
features_per_column = len(percentiles) + 4

def extract_features(xs):
    result = np.ndarray(shape=(xs.shape[1], features_per_column),  dtype='float64')

    for i in xrange(xs.shape[1]):
        x = xs[:, i]
        
        result[i, 0] = np.sum(x == 0.0)
        result[i, 1] = np.mean(x == 0.0)
        
        x_ = x[x != 0.0]
        
        if result[i, 1] < 1.0:
            result[i, 2] = np.mean(x_)
            result[i, 3] = np.std(x_)

            result[i, 4:] = np.percentile(x, percentiles) - np.mean(x)
        else:
            result[i, 2:] = 0.0
            result[i, 3] = -1.0

    return result.ravel()


def group_n_extract(data):
    grouped = data.groupby(by=['run', 'lumiBlock'], sort=False)
    
    run_lumi = list()
    stats = list()
    
    for i in grouped.groups.keys():
        run, lumi_block = i
        idx = np.array(grouped.groups[i])

        lumidata = data.iloc[idx][features].values
        fs = extract_features(lumidata)

        run_lumi.append((run, lumi_block))
        stats.append(fs)
    
    run_lumi = np.array(run_lumi, dtype='int32')
    stats = np.array(stats)

    return run_lumi, stats

In [7]:
idx, X = group_n_extract(data)

In [8]:
idx

array([[146944,    225],
       [146644,   1165],
       [146644,    544],
       ..., 
       [147115,    507],
       [146437,    155],
       [146514,    445]], dtype=int32)

In [9]:
del data

In [10]:
features_per_column

13

In [11]:
m = X.shape[1]

In [12]:
jet_features = range(7 * features_per_column) + [m - 1]
pho_features = range(7 * features_per_column, 13 * features_per_column) + [m - 1]
mou_features = range(13 * features_per_column, 20 * features_per_column) + [m - 1]

In [13]:
X_photon = X[:, pho_features]

In [14]:
X_photon.shape

(33990, 79)

In [15]:
import json

with open(index_file, 'r') as f:
    labels = json.load(f)

In [16]:
y = np.zeros(shape=idx.shape[0], dtype='int32')

for i, (run, lumi_block) in enumerate(idx):
    if str(run) not in labels:
        continue

    run_idxs = labels[str(run)]

    for a, b in run_idxs:
        if a <= lumi_block <= b:
            y[i] = 1
            continue

In [17]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.grid_search import GridSearchCV

In [18]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_jobs=-1)

In [21]:
skfolds = StratifiedKFold(y, n_folds=10)

param_grid = {
    'max_depth' : [2, 5, 9, 15, 20],
    'n_estimators' : [50, 100, 250]
}

grid_search = GridSearchCV(clf, param_grid=param_grid, cv = skfolds, scoring='roc_auc', n_jobs=1)

In [22]:
grid_search.fit(X_photon, y)

GridSearchCV(cv=sklearn.cross_validation.StratifiedKFold(labels=[1 1 ..., 1 1], n_folds=10, shuffle=False, random_state=None),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 250], 'max_depth': [2, 5, 9, 15, 20]},
       pre_dispatch='2*n_jobs', refit=True, scoring='roc_auc', verbose=0)

In [23]:
grid_search.best_score_

0.81916469308131568